In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import random
import json
from tqdm import tqdm
from datetime import datetime

In [2]:
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk import word_tokenize
import re

In [3]:
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display

In [4]:
random.seed(3)
np.random.seed(3)

## Load and fix CSV Files
- strip redundant whitespaces
- combine splitted columns in lyrics
- drop dups

In [5]:
df_train = pd.read_csv('lyrics_train_set.csv', header=None)
df_test = pd.read_csv('lyrics_test_set.csv', header=None)

def fix_csv_lyrics_rows(df):
    df_concat = []
    for idx, row in df.iterrows():
        lyrics = [c for c in row[2:].tolist() if c == c]
        df_concat.append([row[0], row[1], ','.join(lyrics)])
    df = pd.DataFrame(df_concat, columns=['Artist', 'Song', 'Lyrics'])
    df['Song'] = df['Song'].apply(lambda x: x.strip())
    df['Artist'] = df['Artist'].apply(lambda x: x.strip())
    df['Lyrics'] = df['Lyrics'].apply(lambda x: x.strip())
    df = df.drop_duplicates()
    return df

df_train = fix_csv_lyrics_rows(df_train)
df_test = fix_csv_lyrics_rows(df_test)

### Split train to train-validaiton

In [6]:
df_train, df_val = train_test_split(df_train, test_size=0.1, random_state=3)

### Load W2Vec Model

In [7]:
print('loading w2v model')
wv = api.load('glove-wiki-gigaword-300')

loading w2v model


### Pre-process lyrics

In [8]:

def preprocess_text(text):
    """
    remove special chars for reducing sparsity (less OutOfVocabulary words)
    """
    allowed_chars_regex = '[^A-Za-z0 & \']+'
    text = text.lower()
    text = re.sub(allowed_chars_regex, '', text)
    words = word_tokenize(text)
    words = [w for w in words if "'" not in w]
    return words


def add_eos_token(wv):
    """
    change & token to represent EOS token
    """
    n_words = len(wv.index2word)
    emb_dim = wv.vector_size
    empty_vectors = np.array([[0]] * n_words)
    wv.vectors = np.concatenate((wv.vectors, empty_vectors), axis=1)
    eos_emb = np.array([0] * (emb_dim + 1))
    eos_emb[-1] = 1
    wv.vectors[wv.index2word.index('&')] = eos_emb


def w2vec_lookup(wv, w, pad_vec):
    if w in wv.vocab:
        return wv.vectors[wv.vocab[w].index]
    else:
        return pad_vec


def convert_lyrics_to_embeddings(wv, lyrics, songs, artists):
    """
    conver lyrics tokens to embeddings using pretrained word2vec model.
    """
    emb_dim = wv.vector_size
    pad_vec = np.array([0] * (emb_dim + 1))
    all_lyrics_embeddings = []
    all_lyrics_tokens = []
    for song_lyrics, song_name, artist in zip(lyrics, songs, artists):
        lyrics_tokens = preprocess_text(song_lyrics)
        all_lyrics_embeddings.append(np.array([w2vec_lookup(wv, w, pad_vec) for w in lyrics_tokens]))
        all_lyrics_tokens.append(lyrics_tokens)
    return all_lyrics_embeddings, all_lyrics_tokens

In [9]:
print('adding EOS token')
add_eos_token(wv)

adding EOS token


## Extract MIDI- feature vectors for songs

In [10]:


def extract_midi_embs_for_song(midi_data, n_sentences):
    """
    helper function - create embedding for 1 song.
    fv include global features (song based) + local features (sentence-based)
    """
    pitch_dist = midi_data.get_pitch_class_histogram().tolist()
    bpm = midi_data.estimate_tempo()
    global_vec = np.array(pitch_dist + [bpm])
    fs = (n_sentences / midi_data.get_end_time())
    chroma_vector = np.array(midi_data.get_chroma(times=np.arange(0, midi_data.get_end_time(), 1. / fs)).tolist())
    
    chroma_vector = chroma_vector.transpose()
    
    if len(chroma_vector) > n_sentences:
        chroma_vector = chroma_vector[:-1]
    
    global_vec = np.tile(global_vec, (n_sentences,1))
    return np.concatenate((global_vec, chroma_vector), axis=1)
    

def extract_midi_embs_for_df(df,  mode, reg=None):
    
    def extract_key_from_fname(fname):
        artist, song = fname.split('_-_')[0].lower().replace('_', ' '), fname.split('_-_')[1].lower().replace('_', ' ')[:-4]
        return artist, song
    
    MIDI_DIR_PATH = 'midi_files'
    filenames = os.listdir(MIDI_DIR_PATH)
    filekeys = [extract_key_from_fname(fname) for fname in filenames]
    midi_files_map = dict(zip(filekeys, filenames))
    
    all_midi_embs = {}
    regressor_data = []
    for idx, row in tqdm(df.iterrows()):
        try:
            artist = row['Artist']
            song = row['Song'] 
            filename = midi_files_map[(artist, song)]
            midi_data = pretty_midi.PrettyMIDI(MIDI_DIR_PATH + "/" + filename)

            pitch_dist = midi_data.get_pitch_class_histogram().tolist()
            
            if mode == 'test':
                n_sentences = int(reg.predict(np.expand_dims(np.array([midi_data.get_end_time(), midi_data.estimate_tempo(), *pitch_dist]),0))[0])
            else: # train mode - we have the lyrics
                n_sentences = row['Lyrics'].count('&') + 1
                regressor_data.append([n_sentences, midi_data.get_end_time(), midi_data.estimate_tempo(), *pitch_dist])

            midi_emb = extract_midi_embs_for_song(midi_data, n_sentences)
            all_midi_embs[(artist, song)] = midi_emb
        except Exception as e:
#             print(e)
            continue
    return all_midi_embs, regressor_data

    

### Create LR Model
Create predictor for num of sentences in song using Logistic Regression

In [11]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import json
from sklearn.model_selection import train_test_split

def create_x_y_for_training(regressor_data):
    """
    """
    X = []
    Y = []
    for regressor_datum in regressor_data:
        x, y = regressor_datum[1:], regressor_datum[0]
        X.append(x)
        Y.append(y)
    return np.array(X), np.array(Y)

In [12]:
def filter_songs_without_midi_files(df, all_midi_embs_glob_train):
    """
    remove from train songs with damaged midi files (or without any)
    """
    artist = df['Artist'].tolist()
    song = df['Song'].tolist()
    df['key'] = list(zip(artist, song))
    df_filtered = df[df['key'].isin(list(all_midi_embs_glob_train.keys()))]
    return df_filtered

### Run Embeddings creation flow (Lyrics + midi features)

In [13]:
midi_embs_train, regressor_data_train = extract_midi_embs_for_df(df_train, 'train')
midi_embs_val, regressor_data_val = extract_midi_embs_for_df(df_val, 'train')

X, Y = create_x_y_for_training([*regressor_data_train, *regressor_data_val])
reg = LinearRegression().fit(X, Y)

all_midi_embs_test, _ = extract_midi_embs_for_df(df_test, 'test', reg)

df_train_filtered = filter_songs_without_midi_files(df_train, midi_embs_train)
df_val_filtered = filter_songs_without_midi_files(df_val, midi_embs_val)

train_lyrics = df_train_filtered['Lyrics'].tolist()
train_songs = df_train_filtered['Song'].tolist()
train_artist = df_train_filtered['Artist'].tolist()

val_lyrics = df_val_filtered['Lyrics'].tolist()
val_songs = df_val_filtered['Song'].tolist()
val_artist = df_val_filtered['Artist'].tolist()

train_lyrics_embeddings, train_lyrics_tokens = convert_lyrics_to_embeddings(wv, train_lyrics, train_songs, train_artist)
val_lyrics_embeddings, val_lyrics_tokens = convert_lyrics_to_embeddings(wv, val_lyrics, val_songs, val_artist)

1it [00:00,  3.21it/s]/home/heldman/.conda/envs/DL/lib/python3.6/site-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)
538it [02:22,  3.77it/s]
60it [00:15,  3.78it/s]
5it [00:01,  3.96it/s]


### Prepeare Data for Keras model
the next function aggregates the data for the keras model. it creates data of the form of [seq, word_label] for training and validation

In [14]:
train_vocab = {w:idx for idx, w in enumerate(set([w for song in [*train_lyrics_tokens, *val_lyrics_tokens] for w in song]))} # {w:idx}

In [15]:
w2v_dim = 301
step = 1
seq_len = 10
vocab_size = len(train_vocab)

def prepeare_embs_for_keras(lyrics_embedding_list, lyrics_tokens_list, midi_emb_list):
    """
    the function creates 2 seperate embeddings for lyrics and midi 
    """
#     random.shuffle(lyrics_embedding_list)
    pad_vecs = np.array([[0] * w2v_dim] * (seq_len-1))

    all_lyrics_embeddings_x = []
    all_midi_emb_x = []
    all_lyrics_W_midi_embeddings_x = []
    all_lyrics_labels_y = []
    
    
    for lyrics_embedding, midi_emb, lyrics_tokens in tqdm(zip(lyrics_embedding_list, midi_emb_list, lyrics_tokens_list)):
        lyrics_embeddings_x = []
        midi_emb_x = []
        lyrics_W_midi_embeddings_x = []
        lyrics_labels_y = []
        lyrics_embedding = np.concatenate((pad_vecs, lyrics_embedding), axis=0)
        
        sentence_idx = 1
        for i in range(0, len(lyrics_embedding) - seq_len, step):
            lyrics_emb = np.array(lyrics_embedding[i : i + seq_len])
            lyrics_embeddings_x.append(lyrics_emb)
            
            if lyrics_tokens[i] == '&':
                sentence_idx += 1
            
            midi_emb_tmp = np.mean(midi_emb[sentence_idx-1:sentence_idx+1], axis = 0)
            midi_emb_x.append(midi_emb_tmp)
            
            midi_emb_tmp = np.expand_dims(midi_emb_tmp, 0)
            midi_emb_tmp = np.tile(midi_emb_tmp, (seq_len,1))
            lyrics_w_midi_emb = np.concatenate((lyrics_emb, midi_emb_tmp), axis=1)
            lyrics_W_midi_embeddings_x.append(lyrics_w_midi_emb)
            
            y = np.zeros(vocab_size)
            y[train_vocab[lyrics_tokens[i + 1]]] = 1
            lyrics_labels_y.append(y)
            
        all_lyrics_embeddings_x.extend(lyrics_embeddings_x)
        all_midi_emb_x.extend(midi_emb_x)
        all_lyrics_W_midi_embeddings_x.extend(lyrics_W_midi_embeddings_x)
        all_lyrics_labels_y.extend(lyrics_labels_y)
        

    all_lyrics_embeddings_x = np.array(all_lyrics_embeddings_x)
    all_midi_emb_x = np.array(all_midi_emb_x)
    all_lyrics_W_midi_embeddings_x = np.array(all_lyrics_W_midi_embeddings_x)
    all_lyrics_labels_y = np.array(all_lyrics_labels_y)
                                      
    return all_lyrics_embeddings_x, all_midi_emb_x, all_lyrics_W_midi_embeddings_x, all_lyrics_labels_y

In [16]:
train_x_lyrics, train_x_midi, train_x_lyrics_w_midi, train_y = prepeare_embs_for_keras(train_lyrics_embeddings, train_lyrics_tokens, midi_embs_train.values())
val_x_lyrics, val_x_midi, val_x_lyrics_w_midi, val_y = prepeare_embs_for_keras(val_lyrics_embeddings, val_lyrics_tokens, midi_embs_val.values() )

530it [00:13, 38.09it/s]
59it [00:12,  4.91it/s]


### Build + Train LSTM Model
late_cat param control the architecture to test.

In [17]:
late_cat = True # the Architecture where the concatenate of the midi vectors happens ***after*** the LSTM output
# late_cat = False # the Architecture where the concatenate of the midi vectors happens ***before*** the LSTM output

In [18]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras import Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy
from keras.layers import Concatenate
from tensorflow.keras import regularizers


In [19]:
l2_reg = regularizers.l2(1e-5)
midi_emb_dim = 25
hidden_units = 100
hidden_dim = 100
dropout_ratio = 0.4


def build_model_with_midi_late_cat(midi_emb_dim, lyric_shape):
    lyric_lstm = Sequential()
    lyric_lstm.add(Input(shape = lyric_shape))
    
#     lyric_lstm.add(LSTM(hidden_units, activation="relu"))
    lyric_lstm.add(LSTM(hidden_units, activation="relu", kernel_regularizer=l2_reg))
    
    decoder = Sequential()
    decoder.add(Input(shape = hidden_units + midi_emb_dim))
    decoder.add(Dropout(dropout_ratio))
    
#     decoder.add(Dense(hidden_dim))
    decoder.add(Dense(hidden_dim, kernel_regularizer=l2_reg))
    
    decoder.add(Dropout(dropout_ratio))
    
#     decoder.add(Dense(vocab_size))
    decoder.add(Dense(vocab_size, kernel_regularizer=l2_reg))
    
    decoder.add(Activation('softmax'))
    
    lyrics_input = Input(lyric_shape)
    midi_input = Input(midi_emb_dim)
    
    lyrics_emb = lyric_lstm(lyrics_input)    
    lyric_and_midi = Concatenate()([lyrics_emb, midi_input])
    words_prob = decoder(lyric_and_midi)
    model = Model(inputs=[lyrics_input, midi_input], outputs=words_prob)
    print(model.summary())
    return model


def build_model_with_midi_early_cat(lyric_shape):
    lyric_lstm = Sequential()
    lyric_lstm.add(Input(shape = lyric_shape))
    
#     lyric_lstm.add(LSTM(hidden_units, activation="relu"))
    lyric_lstm.add(LSTM(hidden_units, activation="relu", kernel_regularizer=l2_reg))
    
    lyric_lstm.add(Dropout(0.6))
    
#     lyric_lstm.add(Dense(vocab_size))
    lyric_lstm.add(Dense(vocab_size, kernel_regularizer=l2_reg))
    
    lyric_lstm.add(Activation('softmax'))
    
    
    lyrics_w_midi_input = Input(lyric_shape)
    words_prob = lyric_lstm(lyrics_w_midi_input)
    model = Model(inputs=lyrics_w_midi_input, outputs=words_prob)
    print(model.summary())
    return model


In [20]:
if late_cat:
    lyric_shape = (seq_len, w2v_dim)
    model = build_model_with_midi_late_cat(midi_emb_dim, lyric_shape)
    expr = 'late_cat'
else:
    lyric_shape = (seq_len, w2v_dim + midi_emb_dim)
    model = build_model_with_midi_early_cat(lyric_shape)
    expr = 'early_cat'

logdir = f'./{expr}_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

callbacks=[EarlyStopping(patience=2, monitor='val_loss', mode='min'), tensorboard_callback]
opt = keras.optimizers.Adam()
# compile model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[categorical_accuracy])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 10, 301)]    0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 100)          160800      input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 125)          0           sequential[1][0]                 
                                                                 input_4[0][0]                

In [21]:
# fit model
# , callbacks=[tensorboard_callback]
if late_cat:
    model.fit(x=[train_x_lyrics, train_x_midi], y=train_y, validation_data=([val_x_lyrics, val_x_midi], val_y), batch_size=128, epochs=10, callbacks=[tensorboard_callback])
else:
    model.fit(x=train_x_lyrics_w_midi, y=train_y, validation_data=(val_x_lyrics_w_midi, val_y), batch_size=128, epochs=10, callbacks=[tensorboard_callback])

Epoch 1/10
1275/1275 [==============================] - 39s 30ms/step - loss: 6.8284 - categorical_accuracy: 0.1197 - val_loss: 5.5893 - val_categorical_accuracy: 0.1673
Epoch 2/10
1275/1275 [==============================] - 20s 16ms/step - loss: 5.3473 - categorical_accuracy: 0.1694 - val_loss: 5.4539 - val_categorical_accuracy: 0.1754
Epoch 3/10
1275/1275 [==============================] - 24s 19ms/step - loss: 5.1414 - categorical_accuracy: 0.1826 - val_loss: 5.3315 - val_categorical_accuracy: 0.1857
Epoch 4/10
1275/1275 [==============================] - 42s 33ms/step - loss: 5.0221 - categorical_accuracy: 0.1915 - val_loss: 5.2499 - val_categorical_accuracy: 0.1937
Epoch 5/10
1275/1275 [==============================] - 24s 19ms/step - loss: 4.9334 - categorical_accuracy: 0.1970 - val_loss: 5.2401 - val_categorical_accuracy: 0.1916
Epoch 6/10
1275/1275 [==============================] - 21s 16ms/step - loss: 4.8642 - categorical_accuracy: 0.2027 - val_loss: 5.2285 - val_categoric

## Generate Songs Lyrics

In [24]:

def sample(train_vocab_reversed, preds, wv, pad_vec):
    """
    Sample 1 word given the predictions
    """
#     helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
#     probas = np.random.multinomial(1, preds, 1)
#     print(np.max(probas))
#     next_idx = np.argmax(probas)
    next_idx = np.random.choice(len(preds), size=1, p=preds)[0]
    next_word = train_vocab_reversed[next_idx]
    if next_word in wv.vocab:
        next_word_vector = wv.vectors[wv.index2word.index(next_word)]
    else:
        next_word_vector = pad_vec
    return next_word, next_word_vector

def generate_song(midi_embs, sentences_number, seed_word, late_cat):

    # w2v_dim = 301
    # step = 1
    # seq_len = 5
    # vocab_size = len(train_vocab)
    train_vocab_reversed = {v:k for k,v in train_vocab.items()}

    generated = ''
    
    if late_cat:
        pad_vec = np.array([0] * (300 + 1))
    else:
        pad_vec = np.array([0] * (300 + 1 + midi_emb_dim))
            
    embeddings_context = []
    for i in range(seq_len-1):
        embeddings_context.append(pad_vec)
    word_emb = wv.vectors[wv.index2word.index(seed_word)]
    pad_vec = np.array([0] * (300 + 1))
    
    if late_cat:
        embeddings_context.append(word_emb)
    else:
        word_emb = np.concatenate([word_emb, midi_embs[0]])
        embeddings_context.append(word_emb)
        
    embeddings_context = np.array(embeddings_context)
    embeddings_context = np.expand_dims(embeddings_context, 0)

    generated = seed_word

    #Generate the text
    sentence_idx = 0
    while sentence_idx + 1 < sentences_number:
        #calculate next word
        preds = model.predict([embeddings_context, np.expand_dims(np.zeros(midi_emb_dim), 0)], verbose=0)[0]
        next_word, next_word_vec = sample(train_vocab_reversed, preds, wv, pad_vec)

        #add the next word to the text
        generated += " " + next_word
        if next_word == '&':
            generated += '\n'
            sentence_idx += 1

        # shift the sentence by one, and and the next word at its end
        embeddings_context = np.roll(embeddings_context, axis=1, shift=-1)
        
        if not late_cat:
            next_word_vec = np.concatenate([next_word_vec, midi_embs[sentence_idx]])
            
        embeddings_context[0, -1] = next_word_vec

    #print the whole text
    print(generated)

In [29]:
test_songs_midi_embeddings = list(all_midi_embs_test.values())
sentences_number_list = [len(song) for song in test_songs_midi_embeddings]
seed_words = ['close', 'if', 'dear', 'hiya', 'all']
# print(seed_words)
for midi_embs, sentences_number, seed_word in zip(test_songs_midi_embeddings, sentences_number_list, seed_words):
    print('Song Lyrics:\n')
    generate_song(midi_embs, sentences_number, seed_word, late_cat)
    print('\n\n')

Song Lyrics:

close like they &
 leave the rats back in away wo &
 slavery the lawyers attend zone and i adown blackbird to long more &
 ooh ce midnight jag soir &
 roof hurting sister yo hey &
 severed the this lot a love &
 man you went back inside &
 hang off on &
 someday i snuggled a cassette shit &
 oh i blue &
 superfly ruined that morning &
 broke double street as soft but something &
 car and say ever good more &
 baby i miss you &
 there i changed blind &
 it drank they to watch my aha &
 you never gon na be tiny goodbye &
 oh time of the chop &
 everyone have on &
 when they lost my bones pregnant we done can had to spin into and morning while your eye &
 say you pull me crazy &
 you any son it killing motherfucker auf and slashin &
 feelin was have to hei &
 lawabiding lido out is a pox tiffanytwisted &
 santa says you could get to diggy do up back to always to my um &
 so he gon na be my breaking behind one night of her basement &
 and i need all it will hide &
 you ai all

In [26]:
test_songs_midi_embeddings = list(all_midi_embs_test.values())
sentences_number_list = [len(song) for song in test_songs_midi_embeddings]
seed_words = ['love', 'love', 'love', 'love', 'love']
# print(seed_words)
for midi_embs, sentences_number, seed_word in zip(test_songs_midi_embeddings, sentences_number_list, seed_words):
    print('Song Lyrics:\n')
    generate_song(midi_embs, sentences_number, seed_word, late_cat)
    print('\n\n')

Song Lyrics:

love were undesirable &
 so soulful motherfuckers &
 with it back on here &
 when there disappointments again &
 i think something so way &
 my small she still done me yeah &
 whenever you got a part &
 a boy as the workin is hoodlums &
 being the street under to mistake teasing balls &
 and and baby if she rap anothin come &
 she knows i racing they salvation key &
 how we is feed to without door through &
 now someone shining up the jaw &
 but you ca answer and by her yet &
 i die in they incense &
 in your kiss &
 so a things ooh i think i gon na na na na na na na na na na na na na na na na na na na na na oh &
 and bring me to love i call &
 sweet home leave me out &
 your load can the night stand forever &
 darling sure got believing nigga &
 sitting of hell deep sign to totaled complain mony five hand china with all passion &
 of not iesha and arrow the retrace and now and fate &
 since are so forget you guide it them everywhere like she and happy &
 she standing her

In [27]:
test_songs_midi_embeddings = list(all_midi_embs_test.values())
sentences_number_list = [len(song) for song in test_songs_midi_embeddings]
seed_words = ['baby', 'baby', 'baby', 'baby', 'baby']
# print(seed_words)
for midi_embs, sentences_number, seed_word in zip(test_songs_midi_embeddings, sentences_number_list, seed_words):
    print('Song Lyrics:\n')
    generate_song(midi_embs, sentences_number, seed_word, late_cat)
    print('\n\n')

Song Lyrics:

baby love pressed that leave this weeks to hide &
 tomorrow is a stuff to &
 i see your side yes and me i just like your empty time &
 i should gon na be rose &
 we took &
 it there these use it hell for &
 i month in my soul &
 you foe makin on my mouth &
 you tell me the diggity sound &
 and look is a funeral &
 that just not comes &
 but when i joke &
 the propers song &
 enough it your rent back two problem &
 i lose but time to need her girl your song is you breeze the yearning with your eyes for her &
 it as the shadows to a song there &
 ey twistin in swords anymore her eat with us &
 kiss it into the balls that paintings are further &
 and got through i jiggy to seem at our brain &
 i sorry i like my diamonds of the basis &
 and he nothing mean a canopy i knocks to pass run you &
 after i in this right &
 such you it a cough &
 &
 i go here sign to the past &
 and not you see the shit another fuck and breakers &
 my ticket who crews it said to love somebody &
 to 

In [30]:
test_songs_midi_embeddings = list(all_midi_embs_test.values())
sentences_number_list = [len(song) for song in test_songs_midi_embeddings]
seed_words = ['god', 'god', 'god', 'god', 'god']
# print(seed_words)
for midi_embs, sentences_number, seed_word in zip(test_songs_midi_embeddings, sentences_number_list, seed_words):
    print('Song Lyrics:\n')
    generate_song(midi_embs, sentences_number, seed_word, late_cat)
    print('\n\n')

Song Lyrics:

god the way &
 yeah and it tries to seem &
 oh kiss in my eyes &
 girl tell me what i gon na be &
 &
 not said you never go &
 &
 i got around where they going by &
 what you told &
 really hard to lean again &
 someday i help her &
 now why ai sorry &
 i through her shines but things mistaken &
 once &
 i wo to like a breeze together in all king close &
 idiots slam away &
 recognize you wo feed &
 i kill to do you &
 you know they will just for my pretty crying &
 i know as the same bandits so what have i wastin now &
 the season lung book &
 i sang a time you four secrets &
 was a jimi flow there you darlin &
 we make a bridge of what &
 no did suspect when we diggity on the street &
 west bump gon na rap love attracted &
 and nothing big hot dick wish &
 and the pennsylvania takes up me closes &
 put the rhythm to the president burns everybody &
 no good not to conscience rocking &




Song Lyrics:

god me all my friends &
 and &
 ooh do you climb ever not &
 but your